<a href="https://colab.research.google.com/github/renarepenning/VideoGameRecommender/blob/main/ScrapingIGDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If we can't figure out how to use the API could we just scrape it?
*   This example is able to scrape everything except the keywords
*   Not sure if we can have it scrape everything under https://www.igdb.com/games/
*   I think it works for all games tho








In [ ]:
!pip3 install lxml
import requests
from bs4 import BeautifulSoup 
import re
import pandas as pd
from IPython.core.display import display, HTML
import datetime as dt
import numpy as np
import os

In [60]:
# load the website and set up beautiful soup
url = "https://www.igdb.com/games/forza-horizon-5"
# needed in order for the server to think we're human so we can access the webpage for scraping
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
r = requests.get(url, headers=headers)
soup = BeautifulSoup(r.content,  "html.parser")

In [68]:
# get title of the game and remove Edit
title = soup.find(class_ = 'banner-title').text
title = title[:-4]

'Forza Horizon 5'

In [61]:
# get everything from the maininfo class (Id, release date, publisher, developer)
main_info = soup.find(class_ = 'optimisly-game-maininfo')

IGDB_Id = main_info.find('span').text
release_dates = main_info.find_all('time') #deal with this later
developer = main_info.find('a', class_="block").text
publisher = main_info.find_all('a', class_="block")[-1].text

In [73]:
# get extrainfo1 (game modes and genres)
next_segment = soup.find(class_ = 'optimisly-game-extrainfo1')

#modes, genres
extra_info1 = next_segment.find_all('a')

# convert modes and genres to text
i=0
mode_genre = []
while i < len(extra_info1):
  mode_genre.append(extra_info1[i].text)
  i = i+1

In [ ]:
game_mode = []
game_genre = []
modes = ["Battle Royale", "Co-operative", "Massively Multiplayer Online (MMO)", "Multiplayer", "Single player", "Split screen"]
genres = ["Point-and-click", "Fighting", "Shooter", "Music", "Platform", "Puzzle", "Racing", "Real Time Strategy (RTS)", 
          "Role-playing (RPG)", "Simulator", "Sport", "Strategy", "Turn-based strategy (TBS)", "Tactical", "Quiz/Trivia",
          "Hack and slash/Beat ‘em up", "Pinball", "Adventure", "Arcade", "Visual Novel", "Indie", "Card & Board Game", "MOBA"]

# determine which attributes are modes and which are genres and add them to the appropriate list
for elm in mode_genre:
  if elm in modes:
    game_mode.append(elm)
  else:
    game_genre.append(elm)

game_genre

['Racing', 'Sport']

In [ ]:
# get the next set of extra info
next_segment2 = soup.find(class_ = 'optimisly-game-extrainfo2')
extra_info2 = next_segment2.find_all('a')

# convert all extra info to text and remove game engine
i=0
info_2 = []
while i < len(extra_info2):
  info_2.append(extra_info2[i].text)
  i = i+1
info_2 = info_2[:-1]

['Open world', 'Forza Horizon', 'First person', 'Third person', 'Forza', 'ForzaTech']
['Open world', 'Forza Horizon', 'First person', 'Third person', 'Forza']


In [75]:
game_perspective = []
game_themes = []
game_series = []
perspectives = ["Auditory", "Bird view / Isometric", "First person", "Side view", "Text", "Third person", "Virtual Reality"]
themes =["Fantasy", "Thriller", "Science Fiction", "Action", "Horror", "Survival", "Historical",
         "Stealth", "Business", "Comedy", "Drama", "Non-fiction", "Educational", "Sandbox", "Kids",
         "Open world", "Warfare", "4X (explore, expand, exploit, and exterminate)", "Eroitc", "Mystery", "Party", "Romance"]

# deterine wheter each attribute is a perspective, theme, or series
for elm in info_2:
  if elm in perspectives:
    game_perspective.append(elm)
  elif elm in themes:
    game_themes.append(elm)
  else:
    game_series.append(elm)

In [47]:
# get the user and critic rating
get_rating = soup.find_all('text')
get_rating

i=0
rating_info = []
while i < len(get_rating):
  rating_info.append(get_rating[i].text)
  i = i+1
member_rating = rating_info[0]
critic_rating = rating_info[2]

90
94


In [76]:
# consolidate data and add to data frame
data = {'title': title, 'developer':[developer], 'publisher': [publisher], 'game mode':[game_mode], 'genre':[game_genre], 'themes': [game_themes], 'series': [game_series], 'member rating': [member_rating], 'critic rating': [critic_rating]}

igdb_df = pd.DataFrame(data)
igdb_df

,title,developer,publisher,game mode,genre,themes,series,member rating,critic rating
0,Forza Horizon 5,Playground Games,Xbox Game Studios,"[Multiplayer, Single player]","[Racing, Sport]",[Open world],"[Forza Horizon, Forza]",90,94
